In [ ]:
import pandas as pd
import numpy as np
avaliacao = pd.read_csv('dados/avaliacao.csv')
produtor = pd.read_csv('dados/produtor_crm.csv')
movimento = pd.read_csv('dados/movimento.csv')

In [ ]:
from sklearn import preprocessing

ids = [x for x in range(1,100)]

df_pivot = produtor
df_pivot['b'] = 0

for coo_filtro in ids:

    df_pivot = produtor[['vlr_potencial_fat','safra', 'vlr_realizado_fat', 'vlr_potencial_rec', 'vlr_realizado_rec','indice_safra', 'ind_fat', 'ind_rec']].loc[(produtor.cod_produtor==coo_filtro) & (produtor.safra != '2019/2020') & (produtor.safra != '2020/2020') & (produtor.safra != '2020/2021')]
    df_pivot = df_pivot.sort_values('safra')
    df_pivot = df_pivot.set_index('safra').reset_index()
    df_pivot_temp = pd.DataFrame()
    df_pivot_temp['vlr_fat'] = preprocessing.scale(df_pivot['vlr_realizado_fat'])
    df_pivot_temp['vlr_rec'] = preprocessing.scale(df_pivot['vlr_realizado_rec'])

    def best_fit(X,Y):
        
        try:
            xbar = sum(X)/len(X)
            ybar = sum(Y)/len(Y)
            n = len(X) # or len(Y)


            numer = sum([xi*yi for xi,yi in zip(X, Y)]) - n * xbar * ybar
            denum = sum([xi**2 for xi in X]) - n * xbar**2
            b = numer / denum

        except:
            b=0

        #print('best fit line:\ny = {:.2f} + {:.2f}x'.format(a, b))

        return b

    X = df_pivot['indice_safra']
    Y = df_pivot_temp['vlr_fat']
    b = best_fit(X,Y)
    df_pivot['b'] = b
    
df_pivot.head().T

In [ ]:
avaliacao.head().T
print(avaliacao.columns, produtor.columns, movimento.columns)

In [ ]:
#def tenta_conv(valor):
#    try:
#        if valor == '(null)':
#            valor = 99/99/9999
#            return pd.to_datetime(valor, infer_datetime_format=True)
#        else: 
#            valor = pd.to_datetime(valor, infer_datetime_format=True)
#    except:
#        return pd.to_datetime(valor, infer_datetime_format=True)
#
#avaliacao.dt_ult_movto = avaliacao.dt_ult_movto.apply(tenta_conv)

In [ ]:
avaliacao = avaliacao[avaliacao.dt_ult_movto != '(null)']

In [ ]:
avaliacao['dt_ult_movto'] = pd.to_datetime(avaliacao['dt_ult_movto'])

In [ ]:
lista_cdn = list(avaliacao['nome-abrev'])

In [ ]:
def tipoSafra(safra):
    try:
        safra = safra.split('/')
        if safra[0]==safra[1]:
            return 'Inverno'
        else:
            return 'Verao'
    except:
        return 'N/D'

def extraiMovimento(codigo, movimento):
    subset = movimento.loc[movimento.cod_produtor==codigo]
    subset['tiposafra'] = movimento.safra.apply(tipoSafra)
    #Caso o produtor tenha mais de uma plantação a função abaixo o agrupa
    datag = subset.groupby(['tiposafra','Codcult','safra']).agg(Recebimento=('qde_tot_rec','sum'),
                                                                n_propriedades=('cod_produtor','count'),
                                                                Faturamento=('vlr_total_fat','sum'),
                                                                Area=('area_pot_int','sum'))
    datag.reset_index(inplace=True)
    #datag.loc[datag.Area == 0, 'Area'] = np.nan
    #datag.Area = datag.Area.fillna(datag.Area.median())
    datag['codigo'] = codigo
    display(datag)

avaliacao.loc[avaliacao.cdn_cliente==10].cdn_cliente.apply(lambda x: extraiMovimento(x, movimento))
print('fim')

In [ ]:
dt_final = pd.DataFrame()

def tipoSafra(safra):
    try:
        safra = safra.split('/')
        if safra[0]==safra[1]:
            return 'Inverno'
        else:
            return 'Verao'
    except:
        return 'N/D'

def extraiMovimento(codigo, movimento):
    subset = movimento.loc[movimento.cod_produtor==codigo]
    subset['tiposafra'] = movimento.safra.apply(tipoSafra)
    #Caso o produtor tenha mais de uma plantação a função abaixo o agrupa
    datag = subset.groupby(['tiposafra','Codcult','safra']).agg(Recebimento=('qde_tot_rec','sum'),
                                                                n_propriedades=('cod_produtor','count'),
                                                                Faturamento=('vlr_total_fat','sum'),
                                                                Area=('area_pot_int','sum'))
    datag.reset_index(inplace=True)
    #datag.loc[datag.Area == 0, 'Area'] = np.nan
    #datag.Area = datag.Area.fillna(datag.Area.median())
    datag['codigo'] = codigo
    #display(datag)
    print(codigo)
    
    return datag


for cdn in lista_cdn:
    
    dt_temp = avaliacao.loc[avaliacao.cdn_cliente==cdn].cdn_cliente.apply(lambda x: extraiMovimento(x, movimento))
    dt_final = pd.concat([dt_final, dt_temp])
    
print('fim')

In [ ]:
dt_final

In [ ]:
# sample points 
X = [0, 5, 10, 15, 20]
Y = [0, 7, 10, 13, 10]

# solve for a and b
def best_fit(X, Y):

    xbar = sum(X)/len(X)
    ybar = sum(Y)/len(Y)
    n = len(X) # or len(Y)

    numer = sum([xi*yi for xi,yi in zip(X, Y)]) - n * xbar * ybar
    denum = sum([xi**2 for xi in X]) - n * xbar**2

    b = numer / denum
    a = ybar - b * xbar

    print('best fit line:\ny = {:.2f} + {:.2f}x'.format(a, b))

    return a, b

# solution
a, b = best_fit(X, Y)
#best fit line:
#y = 0.80 + 0.92x

# plot points and fit line
import matplotlib.pyplot as plt
plt.scatter(X, Y)
yfit = [a + b * xi for xi in X]
plt.plot(X, yfit)
plt.show()
print(b)